# Data 1050 SQL Project

## Create Database

In [1]:
pip install mysql-connector-python # type: ignore

Note: you may need to restart the kernel to use updated packages.


In [2]:
import mysql.connector # type: ignore
 
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password= "7@u*sqJX"   #REPLACE THIS WITH THE PASSWORD YOU SET
)
 
print(mydb)
 
if mydb.is_connected():
    print("CONNECTION SUCCESSFUL")

CONNECTION SUCCESSFUL


In [3]:
#create a database
mycursor = mydb.cursor()
mycursor.execute("DROP DATABASE IF EXISTS data1050SQLProject")
mycursor.execute("CREATE DATABASE data1050SQLProject")

In [4]:
mycursor = mydb.cursor()
mycursor.execute("SHOW DATABASES")

for x in mycursor:
  print(x)

('data1050f24',)
('data1050SQLProject',)
('information_schema',)
('mysql',)
('performance_schema',)
('sys',)


### Connecting to a database and showing tables

In [5]:
#connect to a specific database
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password= "7@u*sqJX", #REPLACE THIS WITH YOUR PASSWORD 
  database = "data1050SQLProject" #connecting to testDatabase
)

In [6]:
mycursor = mydb.cursor()

mycursor.execute("SHOW TABLES")

for x in mycursor:
  print(x)

In [8]:
import pandas as pd # type: ignore

In [9]:
pip install fsspec # type: ignore

Note: you may need to restart the kernel to use updated packages.


### Add Tables